In [1]:
%%capture
%run 1-Register\ File\ Example.ipynb
%run 2-Defining\ an\ APB\ Model\ in\ Python.ipynb
%run 3-Verifying\ The\ Register\ File.ipynb

In [2]:
# For some reason logging in this notebook doesn't work unless I explicitly set
# the handler, I suspect this is due to some interaction with the above
# notebooks being run (which configure their own loggers)
# Adapted from https://github.com/ipython/ipykernel/issues/111#issuecomment-237089618

import sys
import logging

# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

First, we'll define a stub DMA module with a basic interface. We'll wire all
the inputs to the CoreIR term module so that the downstream compiler does not
complain about dangling wires.

In [3]:
import mantle


class DMA(m.Circuit):
    """
    Stub DMA module
    """
    io = m.IO(csr=m.In(m.Bits[32]), src_addr=m.In(m.Bits[32]),
              dst_addr=m.In(m.Bits[32]), txfr_len=m.In(m.Bits[32]))

    io.csr.unused()
    io.src_addr.unused()
    io.dst_addr.unused()
    io.txfr_len.unused()

Now we'll define a top module containing two DMA instances that is
parameterized by a mode `pack` or `distributed` which refers to the strategy
for managing the configuration registers for the DMA. In the `pack` mode, the
configuration registers are placed in a single register file that is shared by
the two DMAs. In the `distributed` mode, each DMA has their own register file.

In [4]:
class TopGenerator(m.Generator2):
    def __init__(self, mode="pack"):
        """
        Simple example that instances two stub DMA modules and is paramtrizable
        over distributed versus packed register file
        """

        if mode not in ["pack", "distribute"]:
            raise ValueError(f"Unexpected mode {mode}")

        fields = ["csr", "src_addr", "dst_addr", "txfr_len"]
        data_width = 32
        if mode == "pack":
            addr_width = math.ceil(math.log2(len(fields) * 2))
        else:
            addr_width = math.ceil(math.log2(len(fields)))

        self.name = "Top_" + mode
        if mode == "pack":
            self.io = io = m.IO(apb=APBSlave(addr_width, data_width, 0))
        else:
            self.io = io = m.IO(apb=APBSlave(addr_width, data_width, [0, 1]))

        dmas = [DMA(name=f"dma{i}") for i in range(2)]
        if mode == "pack":
            regs = tuple(Register(name + str(i)) for i in range(2) for name in
                         fields)
            reg_file = RegisterFileGenerator(regs, data_width=32)(name="reg_file")
            for i in range(2):
                for name in fields:
                    m.wire(getattr(reg_file, name + str(i) + "_q"),
                           getattr(dmas[i], name))
            m.wire(io.apb, reg_file.apb)
            for i in range(2):
                for name in fields:
                    m.wire(getattr(reg_file, name + str(i) + "_q"),
                           getattr(reg_file, name + str(i) + "_d"))
        else:
            apb_outputs = {}
            for key, type_ in APBBase(addr_width, data_width).items():
                if type_.is_input():
                    apb_outputs[key] = []
            for i in range(2):
                regs = tuple(Register(name) for name in fields)
                reg_file = RegisterFileGenerator(
                    regs, data_width=32, apb_slave_id=i
                )(name=f"reg_file{i}")
                for name in fields:
                    m.wire(getattr(reg_file, name + "_q"),
                           getattr(dmas[i], name))
                for key, type_ in APBBase(addr_width, data_width).items():
                    if type_.is_output():
                        m.wire(getattr(io.apb, key),
                               getattr(reg_file.apb, key))
                    else:
                        apb_outputs[key].append(getattr(reg_file.apb, key))
                m.wire(getattr(io.apb, f"PSEL{i}"),
                       getattr(reg_file.apb, f"PSEL{i}"))
                for name in fields:
                    m.wire(getattr(reg_file, name + "_q"),
                           getattr(reg_file, name + "_d"))
            for key, values in apb_outputs.items():
                m.wire(getattr(io.apb, key),
                       mantle.mux(values, io.apb.PSEL1))

To test this top module, we can use the same read/write request functions that
we created to test our register file.  Our test bench will be parametrized by
mode, enabling the reuse of tests for both variants of the generated design.

Here's a simple write test

In [5]:
import fault

dma_fields = ["csr", "src_addr", "dst_addr", "txfr_len"]

def test_top_simple_write(mode, num_slaves):
    Top = TopGenerator(mode=mode)
    print(Top, mode)

    tester = fault.Tester(Top, clock=Top.apb.PCLK)
    tester.circuit.apb.PRESETn = 1

    addr_width = len(Top.apb.PADDR)
    data_width = len(Top.apb.PWDATA)
    bus = APBBus(addr_width, data_width, num_slaves)
    for i in range(2):
        for addr, field in enumerate(dma_fields):
            if mode == "pack":
                addr += i * len(dma_fields)
                slave_id = 0
            else:
                slave_id = i
            data = fault.random.random_bv(data_width)
            io, request = make_request(addr, data, addr_width, data_width,
                                       num_slaves, slave_id)

            write(bus, io, request, tester, addr, data)
            if mode == "pack":
                getattr(tester.circuit.reg_file, f"{field}{i}_q").expect(data)
            else:
                getattr(getattr(tester.circuit, f"reg_file{i}"),
                        f"{field}_q").expect(data)
            getattr(getattr(tester.circuit, f"dma{i}"),
                    f"{field}").expect(data)

    tester.compile_and_run(target="verilator", magma_output="coreir-verilog",
                           magma_opts={"verilator_debug": True})

We can test packed mode (1 slave)

In [6]:
test_top_simple_write("pack", 1)

Top_pack(apb: Tuple(PSEL0=In(Bit),PCLK=In(Clock),PRESETn=In(Reset),PADDR=In(Bits[3]),PPROT=In(Bit),PENABLE=In(Bit),PWRITE=In(Bit),PWDATA=In(Bits[32]),PSTRB=In(Bits[4]),PREADY=Out(Bit),PRDATA=Out(Bits[32]),PSLVERR=Out(Bit))) pack


root - INFO - Running tester...
root - INFO - Success!


and distribute mode (2 slaves)

In [7]:
test_top_simple_write("distribute", 2)

Top_distribute(apb: Tuple(PSEL0=In(Bit),PSEL1=In(Bit),PCLK=In(Clock),PRESETn=In(Reset),PADDR=In(Bits[2]),PPROT=In(Bit),PENABLE=In(Bit),PWRITE=In(Bit),PWDATA=In(Bits[32]),PSTRB=In(Bits[4]),PREADY=Out(Bit),PRDATA=Out(Bits[32]),PSLVERR=Out(Bit))) distribute


root - INFO - Running tester...
root - INFO - Success!


We can write a similar test for a sequence of writes followed by a sequence of
reads

In [8]:
def test_top_write_then_reads(mode, num_slaves):
    Top = TopGenerator(mode=mode)

    tester = fault.Tester(Top, clock=Top.apb.PCLK)
    tester.circuit.apb.PRESETn = 1

    addr_width = len(Top.apb.PADDR)
    data_width = len(Top.apb.PWDATA)
    bus = APBBus(addr_width, data_width, num_slaves)
    expected_values = []
    for i in range(2):
        for addr, field in enumerate(dma_fields):
            if mode == "pack":
                addr += i * len(dma_fields)
                slave_id = 0
            else:
                slave_id = i
            data = fault.random.random_bv(data_width)
            expected_values.append(data)
            io, request = make_request(addr, data, addr_width, data_width,
                                       num_slaves, slave_id)
            write(bus, io, request, tester, addr, data)

    for i in range(2):
        for addr, field in enumerate(dma_fields):
            data = expected_values[addr + i * len(dma_fields)]
            if mode == "pack":
                addr += i * len(dma_fields)
                slave_id = 0
            else:
                slave_id = i
            io, request = make_request(addr, data, addr_width, data_width,
                                       num_slaves, slave_id)
            getattr(getattr(tester.circuit, f"dma{i}"),
                    f"{field}").expect(data)
            read(bus, io, request, tester, addr, data)

    tester.compile_and_run(target="verilator", magma_output="coreir-verilog",
                           magma_opts={"verilator_debug": True},
                           flags=["--trace", "-Wno-UNUSED", "-Wno-UNDRIVEN"], 
                           tmp_dir=True)

In [9]:
test_top_write_then_reads("pack", 1)

root - INFO - Running tester...
root - INFO - Success!


and distribute mode (2 slaves)

In [10]:
test_top_write_then_reads("distribute", 2)

root - INFO - Running tester...
root - INFO - Success!
